In [17]:
import numpy as nmp 
import math
import os
import os.path



def is_on_left(arg,xs):
    if arg<=(xs[0]+xs[len(xs)-1])/2:
        return True
    else:
        return False

    
def get_index(arg,xs):
    for i in range(0,len(xs)):
        if (xs[i] > arg):
            return i


def is_equidistant(xs):
    h = round(xs[1]-xs[0],10)
    for i in range(1,len(xs)):
        if round(xs[i]-xs[i-1],10) != h:
            return False 
    return True


def linear_interpolation(arg,xs,ys):
    i = get_index(arg,xs)
    A = (ys[i]-ys[i-1])/(xs[i]-xs[i-1])
    B = ys[i-1]-(A*xs[i-1])
    return (A*arg + B)


def poly_interpolation(arg,xs,ys):
    i = get_index(arg,xs)
    x_matrix = nmp.array([[xs[i-2]*xs[i-2],xs[i-2],1], 
                       [xs[i-1]*xs[i-1],xs[i-1],1], 
                       [xs[i]*xs[i],xs[i],1]])
    
    results = nmp.array([ys[i-2],ys[i-1],ys[i]])
    a, b, c = nmp.linalg.solve(x_matrix, results) 
    return (a*arg*arg + b*arg + c)


def lagrange_interpolation(arg,xs,ys):
    res = 0
    for i in range (0,len(ys)):
        product=1
        for j in range(0,len(xs)):
            if i!=j:
                product *= (arg-xs[j])/(xs[i]-xs[j]) 
        res += ys[i]*product
    return res


def newton_first(arg, xs, ys):
    arg_i = get_index(arg,xs) - 1
    res = ys[arg_i]
    iterator = len(xs) - 1 - arg_i
    diffs = []
    h = xs[1] - xs[0]
    t = (arg-xs[arg_i])/h   
    
    for i in range(iterator):
        diffs.append([])
    for i in range(arg_i,len(xs)-1):
        diffs[0].append(ys[i+1]-ys[i])
    for i in range(1,len(diffs)):
        for j in range(len(diffs[i-1])-1): 
            diffs[i].append(diffs[i-1][j+1] - diffs[i-1][j])
    for i in range(1,iterator):
        acc = 1
        for j in range(i):
            acc*=(t-j)
            if (j!=0):
                acc/=j
        acc *= diffs[i-1][0]
        res += acc
                
    return res
        
    
def newton_second(arg,xs,ys):
    arg_i = get_index(arg,xs)
    res = ys[arg_i]
    iterator = arg_i
    diffs = []
    h = round((xs[1]-xs[0]),10) 
    t = (arg-xs[arg_i])/h 
    
    for i in range(iterator):
        diffs.append([])
    for i in range(0,arg_i):
        diffs[0].append(ys[i+1]-ys[i])
    for i in range(1,len(diffs)):
        for j in range(len(diffs[i-1])-1): 
            diffs[i].append(diffs[i-1][j+1] - diffs[i-1][j])
    for i in range(1,iterator-1):
        acc = 1
        for j in range(i):
            acc*=(t+j)
            if (j!=0):
                acc/=j
        acc *= diffs[i-1][len(diffs[i-1])-i]
        res += acc
                
    return res
    
    
def newton_ddiff(arg, xs, ys, i0, i1, i2):
    
    def ddiff_1st(x1,x0,y1,y0):
        return (y1-y0)/(x1-x0)

    def ddiff_2nd(x2,x1,x0,y2,y1,y0):
        return (ddiff_1st(x2,x1,y2,y1)-ddiff_1st(x1,x0,y1,y0))/(x2-x0)
    
    return ys[i0] + (arg-xs[i0]) * ddiff_1st(xs[i1], xs[i0], ys[i1], ys[i0]) + (arg-xs[i0]) * (arg-xs[i1]) * ddiff_2nd(xs[i2], xs[i1], xs[i0], ys[i2], ys[i1], ys[i0]) ##will this new line work?





print("Введите имя файла для считывания\n")
is_correct = True
try:
    while(True):
        filename = input()
        if os.path.exists(filename):
            break
        else:
            print("Такого файла не существует. Попробуйте еще раз.")
        
    input_file = open(filename,'r')
    string_x = str(input_file.readline())
    string_y = str(input_file.readline())
    input_file.close()
    xs = nmp.asarray(list(map(float,string_x.split())))
    ys = nmp.asarray(list(map(float,string_y.split())))
    
    element_max = nmp.amax(xs)
    element_min = nmp.amin(xs)
    
    if len(xs)!=len(ys):
        print("Количество X не соответствует количеству Y.")
        is_correct = False
except ValueError as e:
    print("В файле найдены нечисловые данные.")
    is_correct = False
    
if is_correct:
    print("Введите значение аргумента, используемого для вычисления приближенного значения функции")
    while(True):
        try:
            arg = float(input())
            if (arg > element_max or arg < element_min):
                print("Число не входит в заданные границы. Попробуйте еще раз.")
            else:
                break
        except ValueError as e:
            print("Введено не число. Попробуйте еще раз.")
            
    print("Линейная интерполяция - " + str(linear_interpolation(arg,xs,ys)))
    print("Квадратичная интерполяция - " + str(poly_interpolation(arg,xs,ys)))
    print("Интерполяция с помощью многочлена Лагранжа - " + str(lagrange_interpolation(arg,xs,ys)))
    print("Интерполяция с помощью неравноотстоящих узлов (0,1,2) - " + str(newton_ddiff(arg,xs,ys,0,1,2)))
    print("Интерполяция с помощью неравноотстоящих узлов (1,2,3) - " + str(newton_ddiff(arg,xs,ys,1,2,3)))
    
    if (is_equidistant(xs)):
        if (is_on_left(arg,xs)):
            print("Интерполяция по первой формуле Ньютона - " + str(newton_first(arg,xs,ys)))
        else:
            print("Интерполяция по второй формуле Ньютона - " + str(newton_second(arg,xs,ys)))
    else:
        print("Функция не содержит в себе равноотстоящих точек")



Введите имя файла для считывания

Lab3Input.txt
Введите значение аргумента, используемого для вычисления приближенного значения функции
1.557
Линейная интерполяция - 5.2529499999999985
Квадратичная интерполяция - 5.253268989999999
Интерполяция с помощью многочлена Лагранжа - 5.254844268587183
Интерполяция с помощью неравноотстоящих узлов (0,1,2) - 5.284790799999985
Интерполяция с помощью неравноотстоящих узлов (1,2,3) - 5.236685434999994
Интерполяция по второй формуле Ньютона - 5.253232959103
